In [65]:
from utils.freq_imports import *
from utils.helper import create_dir
from evaluation.kinect.capture_data import get_kinecvtv2_stored_intrinsics
from utils import plotly_wrapper
from ipywidgets import Output, VBox
from utils.perspective_projection import uvd2xyz, xyz2uv
from evaluation.kinect.preprocess import process_frame


In [281]:
# read depth image
root_dir = "../../.."
path_to_kinect_data = Path(f"{root_dir}/data/kinect")

# ----------------------------------------
# NOTE: change this for every user
user = "vihaan"
sequence = 6
# ----------------------------------------
in_seq_dir = f"{path_to_kinect_data}/{user}/{sequence}"
in_color_raw_dir = f"{in_seq_dir}/color_raw"
in_depth_raw_dir = f"{in_seq_dir}/depth_raw"
in_depth_proc_dir = f"{in_seq_dir}/depth_proc"
i_frame = 1
color_raw = cv.imread(f"{in_color_raw_dir}/{i_frame:05d}.png")[:, :, ::-1]
depth_raw = np.load(f"{in_depth_raw_dir}/{i_frame:05d}.npy")

H, W = 424, 512
fx, fy, cx, cy = get_kinecvtv2_stored_intrinsics()
z_near, z_far = 0.6, 1.2   # m
d_near = z_near * 1000; d_far = z_far * 1000    # depth uses mm as units
    
# xyz_crop_center = np.array([0, 0, 0.7])
# color_proc, depth_proc, mask_sil, xyz_crop_center = process_frame(color_raw, depth_raw, d_near, d_far, fx, fy, cx, cy, xyz_crop_center)
depth_proc = np.load(f"{in_depth_proc_dir}/{i_frame:05d}.npy")



In [282]:
hm = go.Heatmap(z=depth_proc, hovertemplate="%{z}<extra></extra>")
fig = go.FigureWidget([hm])
plotly_wrapper.invert_fig_y(fig)
scale = 2
fig.update_layout(width=W*scale, height=H*scale, hoverdistance=1)

# marking order:
# wrist
# thumb: mcp, pip, dip, tip
# index: mcp, pip, dip, tip
# middle: mcp, pip, dip, tip
# ring: mcp, pip, dip, tip
# pinky: mcp, pip, dip, tip
k_uvd_marked = []
out = Output()

@out.capture(clear_output=False)
def mark_points(trace, points, selector):
    v, u = points.point_inds[0]
    d = depth_proc[v, u] / 1000
    uvd = np.array([u, v, d])
    # print(uvd)
    k_uvd_marked.append(uvd)
    
    fig.add_trace(go.Scatter(x=[u], y=[v], mode="markers", marker=dict(size=5, color="red"), hovertemplate="<extra></extra>", showlegend=False))

fig.data[0].on_click(mark_points)

VBox([fig, out])

    'data': [{'hovertemplate': '%{z}<extra></extra>',
              'type': 'hea…

In [283]:
k_marked = uvd2xyz(np.array(k_uvd_marked), fx, fy, cx, cy)

out_dir = f"{root_dir}/output/kinect/marked_keypoints/{user}/{sequence}"; create_dir(out_dir, True)
plotly_wrapper.invert_fig_y(fig)    # is this required again?
fig.write_html(f"{out_dir}/marked_keypoints.html")
np.save(f"{out_dir}/k_marked.npy", k_marked)